In [ ]:
using CairoMakie;
using CSV, DataFrames

using LilGuys
using Arya
import DensityEstimators as DE

In [ ]:
using StatsBase

In [ ]:
using PythonCall

In [ ]:
using Printf

In [ ]:
agama = pyimport("agama")

## File loading

In [ ]:
potential = agama.Potential("../agama/potentials/EP2020.ini")

In [ ]:
disk_poetntial = agama.Potential(potential[1], potential[2], potential[0])

In [ ]:
model_dir = ENV["DWARFS_ROOT"] * "/analysis/sculptor/1e7_V31_r3.2/orbit_mean/"

In [ ]:
starsname = "plummer_rs0.20"

In [ ]:
starsfile = "../stars/$starsname/probabilities_stars.hdf5"

In [ ]:
prob_df = LilGuys.read_hdf5_table(joinpath(model_dir, starsfile))

In [ ]:
issorted(prob_df.index)

In [ ]:
out = Output(model_dir, weights=prob_df.probability)

In [ ]:
# for DM & stellar animations.
r_max = 200

bins = LinRange(-r_max, r_max, 1000 + 1)

## Stellar disk image

In [ ]:
Nb = length(bins)

In [ ]:
ρ_min = pyconvert(Float64, potential[3].density([r_max, r_max, r_max]))

In [ ]:
Σ_min = pyconvert(Float64, sum(potential[3].density([fill(r_max, Nb) fill(r_max, Nb) bins])))

In [ ]:
ρ_min * length(bins)

In [ ]:
ay = [1,2]
az = [3,5,6]

In [ ]:
y = repeat(ay, outer=3)

In [ ]:
z = repeat(az, inner=2)

In [ ]:
reshape(y, 2, 3)

In [ ]:
reshape(z, 2, 3)

In [ ]:
sum(reshape(z, 2, 3), dims=2)

In [ ]:
function sum_density(disk_potential, bins; cutoff=40, cutoffstrength=3)
    bin_mids = midpoints(bins)
    Nb = length(bin_mids)
    
    Σ_disk = Matrix{Float64}(undef, Nb, Nb)
    for i in 1:Nb
        print("$i/$Nb\r")
        
        x = fill(bin_mids[i], Nb*Nb)
        y = vcat([fill(a, Nb) for a in bin_mids]...)
        z = repeat(bin_mids, Nb)
        r = @. sqrt(x^2 + y^2 + z^2)

        # NOTE--this should be more formalized
        pos = [x z y]
        rho = disk_poetntial.density(pos)
        rho = pyconvert(Array{Float64}, rho)
        if cutoff !== nothing
            rho = @. rho * exp( -(r / cutoff)^cutoffstrength) 
        end
        
        for j in 0:(Nb-1)
            Σ = sum(rho[1+j*Nb:Nb*(j+1)])
            Σ_disk[i, j+1] = Σ

        end
    end

    return Σ_disk
end
        

In [ ]:
function normalized_density(x, h=0.03)
    x = asinh.(x ./ h)
    xm, xh = extrema(x[isfinite.(x)])
    return (x .- xm) ./ (xh .- xm)
end

In [ ]:
function normalized_density2(x, h=0.03)
    x = log10.(x .+ h)
    xm, xh = extrema(x[isfinite.(x)])
    return (x .- xm) ./ (xh .- xm)
end

In [ ]:
color_0 = Arya.COLORS[7] 

In [ ]:
print(color_0)

In [ ]:
Makie.RGBAf(0., 0., 0.)

In [ ]:
Makie.RGBA{Float32}(0.9411765f0,0.89411765f0,0.25882354f0,1.0f0)


In [ ]:
function transparency_map(x) 
    x = x
    Makie.RGBAf(color_0.r, color_0.g, color_0.b * x, x)
end

In [ ]:
Sigma_disk = sum_density(disk_poetntial, bins)

In [ ]:
Sigma_disk_nocut = sum_density(disk_poetntial, bins, cutoff=nothing)

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk_nocut)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density2(Sigma_disk, Σ_min)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

In [ ]:
fig, ax = FigAxis(
    yscale=log10,
    limits=(nothing, nothing, 1, 1e6)
)

stephist!(vec(asinh.(Sigma_disk_nocut,)))
stephist!(vec(asinh.(Sigma_disk,)))
stephist!(vec(log10.(Sigma_disk .+ Σ_min)))
stephist!(vec(log10.(Sigma_disk_nocut .+ Σ_min)))
#
fig

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density2(Sigma_disk_nocut, 10Σ_min)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

## Animate dark matter

In [ ]:
function get_xy2(out, idx)
    # shortcut for hdf5
    idx -= 1
    pos = out.h5file["snap$idx/PartType1/Coordinates"][:, :]
    xy = [ [0, 1, 0] [0, 0, 1]]' * pos
    return xy[1, :], xy[2, :]
end

In [ ]:
function get_xy(out, idx)
    # shortcut for hdf5
    idx -= 1
    x = out.h5file["snap$idx/PartType1/Coordinates"][2, :]
	y = out.h5file["snap$idx/PartType1/Coordinates"][3, :]
    return x, y
end

In [ ]:
function project_points(x, y, bins)
    h1 = fit(Histogram, (x, y), (bins, bins) )
end

In [ ]:
@time x, y = get_xy(out, 7)

In [ ]:
@time x, y = get_xy2(out, 7)

In [ ]:
@time h1 = project_points(x, y, bins)

In [ ]:
extrema.(h1.edges)

In [ ]:
x, y = get_xy(out, 212)
h_end = project_points(x, y, bins)

In [ ]:
colormax = log10(maximum(h1.weights))
colorrange=(0, colormax)

In [ ]:
function make_frame(h, colorrange=colorrange)
    fig = Figure(figure_padding=(0,0,0,0), size=(200,200)) # px scale is x5 so this works for 1000 bins
    ax = Axis(fig[1, 1],
        aspect=DataAspect(),
    )
    

    p = image!(extrema(bins), extrema(bins), log10.(h.weights), colorrange=colorrange)

    image!(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk)))
    hidespines!(ax)
    resize_to_layout!(fig)
    hidedecorations!(ax)
    lines!([-175, -125], [-175, -175], color=:grey)
    text!(-175, -175, text="50 kpc", color=:grey, font=font, fontsize=7.5)

    return Makie.FigureAxisPlot(fig, ax, p)
end

In [ ]:
font = "/astro/dboyea/fonts/Arev.ttf"

In [ ]:
# example last frame

make_frame(h1)

In [ ]:
# example last frame

make_frame(h_end)

In [ ]:
h = project_points(x, y, bins)

In [ ]:
idxs = eachindex(out)

In [ ]:
animation_dir = "./figures/sculptor_dm_ani/"

In [ ]:
mkpath(animation_dir)

In [ ]:
h

In [ ]:
function animate(out, bins, animation_dir)
    idx = eachindex(out)
    for i in idx
        print("animating $i / $(idx[end])\r")
        x, y = get_xy(out, i)
        h = project_points(x, y, bins)
        fig, ax, p = make_frame(h)
        Makie.save(joinpath(animation_dir, "frame_$i.png"), fig)
    end
end

In [ ]:
animate(out, bins, animation_dir)

## Animate stars

In [ ]:
idxs = eachindex(out)

In [ ]:
function get_xy(out, idx)
    # shortcut for hdf5
    snap = out[idx]
    x = snap.positions[2, :]
    y = snap.positions[3, :]
    w = snap.weights
    return x, y, w
end

In [ ]:
function project_points(x, y, w, bins)
    h1 = fit(Histogram, (x, y), weights(w), (bins, bins), )
end

In [ ]:
@time x, y, w = get_xy(out, 1)

In [ ]:
@time h = project_points(x, y, w, bins)

In [ ]:
x, y, w = get_xy(out, 212)
h_end = project_points(x, y, w, bins)

In [ ]:
sum(out.weights)

In [ ]:
Ntot = 5e6

In [ ]:
log10(Ntot)

In [ ]:
colormax = log10(maximum(h.weights))
colorrange = (-log10(Ntot), colormax) # from a single star to the maximum

In [ ]:
function make_frame(h, colorrange=colorrange)
    fig = Figure(figure_padding=(0,0,0,0), size=(200,200)) # px scale is x5 so this works for 1000 bins
    ax = Axis(fig[1, 1],
        aspect=DataAspect(),
    )
    

    p = image!(extrema(bins), extrema(bins), log10.(h.weights), colormap=Arya.get_arya_cmap(), colorrange=colorrange)
    image!(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk)))
    
    hidespines!(ax)
    resize_to_layout!(fig)
    hidedecorations!(ax)
    lines!([-175, -125], [-175, -175], color=:grey)
    text!(-175, -175, text="50 kpc", color=:grey, font=font, fontsize=7.5)

    return Makie.FigureAxisPlot(fig, ax, p)
end

In [ ]:
@time make_frame(h)

In [ ]:
@time make_frame(h_end)

In [ ]:
animation_dir = "./figures/sculptor_stars_ani/"

In [ ]:
mkpath(animation_dir)

In [ ]:
function animate(out, bins, animation_dir)
    idx = eachindex(out)
    for i in idx
        print("animating $i / $(idx[end])\r")
        x, y, w = get_xy(out, i)
        h = project_points(x, y, w, bins)
        fig, ax, p = make_frame(h)
        Makie.save(joinpath(animation_dir, "frame_$i.png"), fig)
    end
end

In [ ]:
animate(out, bins, animation_dir)

In [ ]:
Ntot = 5e6 # conservatively large estimate for number of stars

In [ ]:
sum(out.weights)

In [ ]:
log10(maximum(h1.weights))

In [ ]:
x, y, w = get_xy(out, 1)
h1 = project_points(x, y, w*Ntot, bins)

In [ ]:
log10(maximum(h1.weights))

In [ ]:
x, y, w = get_xy(out, 213)
h = project_points(x, y, w*Ntot, bins)

fig, ax, p = make_frame(h, (0, log10(maximum(h1.weights))) )
Makie.save(animation_dir * "/test.png", fig)

In [ ]:
log10(maximum(h1.weights / Ntot))

In [ ]:
x, y, w = get_xy(out, 213)
h = project_points(x, y, w, bins)

fig, ax, p = make_frame(h, (-6.7, log10(maximum(h1.weights / Ntot)) ))
Makie.save(animation_dir * "/test2.png", fig)

## Animate density profile

### Densities

In [ ]:
function density_axis(gs; kwargs...)
    ax = Axis(gs;
        xlabel="log r / kpc",
        ylabel=L"\log\rho",
        limits=(-2, 2, -10, 1),
        kwargs...
    )

    return ax
end

In [ ]:
function density_axis()
    fig = Figure()
    ax = density_axis(fig[1, 1])

    return fig, ax
end

In [ ]:
prof_path = joinpath(model_dir, "profiles.hdf5")

profiles = LilGuys.read_structs_from_hdf5(prof_path, LilGuys.MassProfile3D) |> LilGuys.structs_to_int_pairs

In [ ]:
fig, ax = density_axis()
i = 1
lines!(profiles[i].second.log_r, log10.(profiles[i].second.rho))
fig

In [ ]:
fig, ax = density_axis()
i = length(profiles)
p = lines!(profiles[i].second.log_r, log10.(profiles[i].second.rho))
fig

In [ ]:
profiles[end-1].first

In [ ]:
fig, ax = density_axis()
framerate=30
i = 1

p = lines!(profiles[i].second.log_r, log10.(profiles[i].second.rho))

N = length(profiles)

record(fig, "sculptor_dm_density.gif", 1:N, framerate = framerate) do i
    print("progress...$i/$N\r")
    
    idx, prof = profiles[i]
    time = out.times[idx] * T2GYR
    
    ax.title = @sprintf "time = %2.2f Gyr" time

    p[1] = [Makie.Point2f(x, y) for (x,y) in zip(prof.log_r, log10.(prof.rho))]
end

fig

### Velocities

In [ ]:
function vcirc_axis(gs; kwargs...)
    ax = Axis(gs;
        xlabel=L"$\log\, r_\textrm{circ}$ / kpc",
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
        limits=(-1.5, 2, 0.5, 1.5),
        kwargs...
    )

    return ax
end

In [ ]:
function vcirc_axis()
    fig = Figure()
    ax = vcirc_axis(fig[1, 1])

    return fig, ax
end

In [ ]:
fig, ax = vcirc_axis()
i = 1
lines!(log10.(profiles[i].second.r_circ), log10.(profiles[i].second.v_circ * V2KMS))
fig

In [ ]:
fig, ax = vcirc_axis()
i = length(profiles)
lines!(log10.(profiles[i].second.r_circ), log10.(profiles[i].second.v_circ * V2KMS))
scatter!(log10.(profiles[i].second.r_circ_max), log10.(profiles[i].second.v_circ_max * V2KMS))
fig

In [ ]:
profiles[end-1].first

In [ ]:
fig, ax = vcirc_axis()
framerate=30
i = 1

p = lines!(log10.(profiles[i].second.r_circ), log10.(profiles[i].second.v_circ * V2KMS))

vmax_points = Observable(Point2f[(log10.(profiles[i].second.r_circ_max), log10.(profiles[i].second.v_circ_max * V2KMS))])
ps = scatter!(vmax_points)

N = length(profiles)

record(fig, "sculptor_dm_vcirc.gif", 1:N, framerate = framerate) do i
    print("progress...$i/$N\r")
    
    idx, prof = profiles[i]
    time = out.times[idx] * T2GYR
    
    ax.title = @sprintf "time = %2.2f Gyr" time

    p[1] = [Makie.Point2f(x, y) for (x,y) in zip(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS))]

    push!(vmax_points[], Makie.Point2f(log10.(prof.r_circ_max), log10.(prof.v_circ_max * V2KMS)))
end

fig

## Stellar profiles 2D

### Densities

In [ ]:
function density_star_axis(gs; kwargs...)
    ax = Axis(gs;
        xlabel="log r / kpc",
        ylabel=L"\log\Sigma_\star",
        limits=(-2, 2, -6, 1),
        kwargs...
    )

    return ax
end

In [ ]:
function density_star_axis()
    fig = Figure()
    ax = density_star_axis(fig[1, 1])

    return fig, ax
end

In [ ]:
?LilGuys.StellarProfile

In [ ]:
prof_path = joinpath(model_dir, "stars", starsname, "stellar_profiles.hdf5")

profiles = LilGuys.read_structs_from_hdf5(prof_path, LilGuys.StellarProfile) |> LilGuys.structs_to_int_pairs

In [ ]:
fig, ax = density_star_axis()
i = 1
lines!(profiles[i].second.log_r, profiles[i].second.log_Sigma)
fig

In [ ]:
fig, ax = density_star_axis()
i = length(profiles)
lines!(profiles[i].second.log_r, profiles[i].second.log_Sigma)
fig

In [ ]:
fig, ax = density_star_axis()
framerate=3
i = 1

p = lines!(profiles[i].second.log_r, profiles[i].second.log_Sigma)

N = length(profiles)

record(fig, "sculptor_star_density.gif", 1:N, framerate = framerate) do i
    print("progress...$i/$N\r")
    
    idx, prof = profiles[i]
    time = out.times[idx] * T2GYR
    
    ax.title = @sprintf "time = %2.2f Gyr" time

    p[1] = [Makie.Point2f(x, y) for (x,y) in zip(prof.log_r, prof.log_Sigma)]
end

fig

### Velocities

In [ ]:
function vcirc_axis(gs; kwargs...)
    ax = Axis(gs;
        xlabel=L"$\log\, r_\textrm{circ}$ / kpc",
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
        limits=(-1.5, 2, 0.5, 1.5),
        kwargs...
    )

    return ax
end

In [ ]:
function vcirc_axis()
    fig = Figure()
    ax = vcirc_axis(fig[1, 1])

    return fig, ax
end

In [ ]:
fig, ax = vcirc_axis()
i = 1
lines!(log10.(profiles[i].second.r_circ), log10.(profiles[i].second.v_circ * V2KMS))
fig

In [ ]:
fig, ax = vcirc_axis()
i = length(profiles)
lines!(log10.(profiles[i].second.r_circ), log10.(profiles[i].second.v_circ * V2KMS))
scatter!(log10.(profiles[i].second.r_circ_max), log10.(profiles[i].second.v_circ_max * V2KMS))
fig

In [ ]:
profiles[end-1].first

In [ ]:
fig, ax = vcirc_axis()
framerate=30
i = 1

p = lines!(log10.(profiles[i].second.r_circ), log10.(profiles[i].second.v_circ * V2KMS))

vmax_points = Observable(Point2f[(log10.(profiles[i].second.r_circ_max), log10.(profiles[i].second.v_circ_max * V2KMS))])
ps = scatter!(vmax_points)

N = length(profiles)

record(fig, "sculptor_dm_vcirc.gif", 1:N, framerate = framerate) do i
    print("progress...$i/$N\r")
    
    idx, prof = profiles[i]
    time = out.times[idx] * T2GYR
    
    ax.title = @sprintf "time = %2.2f Gyr" time

    p[1] = [Makie.Point2f(x, y) for (x,y) in zip(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS))]

    push!(vmax_points[], Makie.Point2f(log10.(prof.r_circ_max), log10.(prof.v_circ_max * V2KMS)))
end

fig